# BVL KI Workshop: Customer Demand Prediction
## Schritt 1: Lade Packages

In [ ]:
# Installation der Libraries im Browser (JupyterLite spezifisch)
import piplite
await piplite.install(['pandas', 'numpy', 'plotly', 'scikit-learn'])

# Standard Libraries für Datenverarbeitung
import sqlite3
import numpy as np
import pandas as pd

# Tools für Exploration & Visualisierung
# import dtale
import plotly.express as px

# Machine Learning Libraries
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

## Schritt 2: Get Data

In [ ]:
# Verbindung zur SQLite Datenbank herstellen
con = sqlite3.connect("data/data.db")

# Tabellen laden
customer = pd.read_sql_query("SELECT * FROM customer", con)
plant = pd.read_sql_query("SELECT * FROM plant", con)
shipment = pd.read_sql_query("SELECT * FROM shipment", con)

## Schritt 3: Data Preparation & Exploration (EDA)

In [ ]:
# 1. Daten zusammenführen (Joins) und Datum konvertieren
shpm = (
    shipment
    .merge(customer, left_on="Ship-to2", right_on="ID", suffixes=("", "_customer"))
    .merge(plant, left_on="Plant", right_on="ID", suffixes=("", "_plant"))
    .assign(del_date=lambda x: pd.to_datetime(x["Delivery_day"], format="%d.%m.%Y"))
)

# 2. Aggregation auf Tagesebene (Zeitreihe erstellen)
shpm_ts = (
    shpm
    .groupby("del_date", as_index=False)
    .agg(TO_total=("GWkg", lambda x: np.nansum(x) / 1000)) # Umrechnung in Tonnen
    .sort_values("del_date")
)

# 3. Interaktive Visualisierung
fig = px.line(
    shpm_ts, x="del_date", y="TO_total",
    markers=True,title=f"Shipments - Calendar Year(s): {shpm_ts['del_date'].dt.year.unique()}")
fig.update_layout(xaxis_title="Delivery Date", yaxis_title="Tonnage Delivered",hovermode="x unified")
fig.show()

## Schritt 4: Train the AI model and predict

In [ ]:
# Basis-DataFrame für das Modell erstellen
df_model = (
    shpm
    .groupby("del_date", as_index=False)
    .agg(TO_total=("GWkg", lambda x: np.nansum(x) / 1000))
)

# Leere Datenpunkte entfernen und sortieren
df_model = df_model.dropna(subset=['del_date']).sort_values('del_date').reset_index(drop=True)

# Feature Engineering: Zeit-Merkmale aus dem Datum extrahieren
def create_features(df):
    df = df.copy()
    df['dayofweek'] = df['del_date'].dt.dayofweek
    df['quarter'] = df['del_date'].dt.quarter
    df['month'] = df['del_date'].dt.month
    df['year'] = df['del_date'].dt.year
    df['dayofyear'] = df['del_date'].dt.dayofyear
    df['weekofyear'] = df['del_date'].dt.isocalendar().week.astype(int)
    return df

df_model = create_features(df_model)

# Train/Test Split (80% Training, 20% Test)
split_ratio = 0.8
split_point = int(len(df_model) * split_ratio)

FEATURES = ['dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'weekofyear']
TARGET = 'TO_total'

# Daten aufteilen
train = df_model.iloc[:split_point].copy()
test = df_model.iloc[split_point:].copy()

X_train = train[FEATURES]
y_train = train[TARGET]
X_test = test[FEATURES]
y_test = test[TARGET]

# XGBoost Regressor initialisieren
reg = GradientBoostingRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=3,
    random_state=42,
    validation_fraction=0.1, # Für Early Stopping Simulation
    n_iter_no_change=50      # Äquivalent zu early_stopping_rounds
)

# Modell trainieren
reg.fit(X_train, y_train)


# Vorhersagen für das Test-Set generieren
test['Prediction'] = reg.predict(X_test)
mae = mean_absolute_error(test[TARGET], test['Prediction'])
print(f"Mean Absolute Error (MAE): {mae:.2f}")

# Detaillierte Ansicht der Testdaten
display(test.head(10))

## Schritt 5: Visualisierung der Ergebnisse

In [ ]:
# Visualisierung der Ergebnisse: Training vs. Actual vs. Prediction

# 1. Labeling der Trainingsdaten
train_plot = train[['del_date', TARGET]].copy()
train_plot['Type'] = 'Training Data'
train_plot = train_plot.rename(columns={TARGET: 'Tons'})

# 2. Labeling der echten Testdaten
test_actual_plot = test[['del_date', TARGET]].copy()
test_actual_plot['Type'] = 'Test Data (Actual)'
test_actual_plot = test_actual_plot.rename(columns={TARGET: 'Tons'})

# 3. Labeling der Vorhersagen
test_pred_plot = test[['del_date', 'Prediction']].copy()
test_pred_plot['Type'] = 'Prediction'
test_pred_plot = test_pred_plot.rename(columns={'Prediction': 'Tons'})

# 4. Alles in einem DataFrame zusammenführen
plot_df = pd.concat([train_plot, test_actual_plot, test_pred_plot], ignore_index=True)

# --- GRAPH ERSTELLEN ---
fig = px.line(
    plot_df, 
    x='del_date', y='Tons', color='Type', 
    title=f'Full Timeline: Training, Test & Forecast (MAE: {mae:.2f})',
    color_discrete_map={
        "Training Data": "gray",        # Kontext (dezent)
        "Test Data (Actual)": "blue",   # Wahrheit
        "Prediction": "red"             # Prognose
    },
    template="plotly_white"
)

# Vertikale Linie zur Trennung von Train und Test hinzufügen
fig.add_vline(x=test['del_date'].min(), line_width=1, line_dash="dash", line_color="black")

fig.update_layout(
    xaxis_title="Date", yaxis_title="Volume (Tons)",
    hovermode="x unified"
)

fig.show()